## Computational Chemistry for Experimentalists
## Module 9: Partial Charges, Oxidation States, and Bond Orders 

Partial atomic charges, oxidation states, and chemical bond orders are central ideas in general chemistry, and is put into practice in computational chemistry. However, there is no single, rigorous, universally acceptable definition of the bond order of a covalent bond, or the charge on an atom in a molecule, or the oxidation state of an atom in a complex. Here we'll show how different definitions all can can build chemical insight.

In [7]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import math
import numpy
import matplotlib.pyplot as plt
from pyscf import gto,scf,dft,cc
from pyscf.tools import cubegen
import py3Dmol
from pyscf.tools import molden 

## Example 1: Water Partial Charges  

The electric dipole moment of an isolated water molecule is an unambiguous observable. The partial atomic charges are not. While the hydrogens should have charge +delta and the oxygen should have charge -2 delta, there's no single "right" answer for delta. 

This first block generates the geometry and HF/3-21G wavefunction of a single water molecule. 

In [4]:
m=Chem.MolFromSmiles('O')
m2=Chem.AddHs(m)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
m3=Chem.MolToMolBlock(m2)
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
m4 = gto.Mole(basis="cc-pvtz")
m4.atom = atoms
m4.build();
mf=scf.RHF(m4)
mf.kernel() 
p = py3Dmol.view(width=200,height=200)
p.addModel(m3, 'sdf')
p.setStyle({'stick':{},'sphere':{"scale":0.1}})
p.render()

converged SCF energy = -76.0560873011978


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

This block computes the dipole moment and two choices of partial charge: 

In [5]:
d=mf.dip_moment()
dmag=numpy.dot(d,d)**0.5
deltaMulliken=mf.mulliken_pop()
deltaLowdin=mf.mulliken_pop_meta_lowdin_ao()
print(deltaLowdin[1])

print('Dipole moment in debye: %.3f'%(dmag))
print('H atom charge delta')
print('%20s  %.3f'%('Mulliken',deltaMulliken[1][1]))
print('%20s  %.3f'%('Lowdin',deltaLowdin[1][1]))

Dipole moment(X, Y, Z, Debye): -0.03874, -2.04261,  0.00000
 ** Mulliken pop  **
pop of  0 O 1s            1.95076
pop of  0 O 2s            0.99981
pop of  0 O 3s            0.23044
pop of  0 O 4s            0.60321
pop of  0 O 2px           0.34013
pop of  0 O 2py           0.40929
pop of  0 O 2pz           0.50807
pop of  0 O 3px           0.60443
pop of  0 O 3py           0.70387
pop of  0 O 3pz           0.83090
pop of  0 O 4px           0.26198
pop of  0 O 4py           0.43889
pop of  0 O 4pz           0.58617
pop of  0 O 3dxy          0.00215
pop of  0 O 3dyz          0.00022
pop of  0 O 3dz^2         0.00057
pop of  0 O 3dxz          0.00000
pop of  0 O 3dx2-y2       0.00007
pop of  0 O 4dxy          0.01076
pop of  0 O 4dyz          0.00163
pop of  0 O 4dz^2         0.00382
pop of  0 O 4dxz          0.00000
pop of  0 O 4dx2-y2       0.00031
pop of  0 O 4f-3          0.00028
pop of  0 O 4f-2          0.00000
pop of  0 O 4f-1          0.00022
pop of  0 O 4f+0          0.00011
p

This block writes the PySCF wavefunction as a .molden file for post-processing by Multiwfn. The Multiwfn package can compute multiple other definitions of partial charge. Here's a brief summary of the computed hydrogen atom partial charges in H2O. Note that the Mulliken charge matches PySCF and that the Lowdin charge is not correct. The charges based on bond order (Hirshfeld, Mulliken) are different from the charges based on electrostatics (Becke, RESP, ADCH, CHELP) :

Hirshfeld:  0.162287

Voroni deformation density: 0.145298

Mulliken:  0.24419

Lowdin:  -0.16996

Becke:  0.356394 

ADCH: 0.356402  

CHELP: 0.364839

Merz-Kollman: 0.361579

CMI: 0.325707 

RESP:  0.361611

In [8]:
with open('H2O.molden', 'w') as f1:
    molden.header(m4, f1)
    molden.orbital_coeff(m4, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

## Example 2: Oxidation States 

The formal oxidation state of a metal atom in a complex doesn't match any known definition of partial charge. Here we compute the Mulliken and Lowden charges on the formally Fe(III) metal atom of high-spin FeCl6(3-). 

In [9]:
m=gto.Mole(atom='Fe 0.0 0.0 0.0; Cl 2.55 0.0 0.0;Cl -2.55 0.0 0.0;Cl 0.0 2.55 0.0; Cl 0.0 -2.55 0.0; Cl 0.0 0.0 2.55; Cl 0.0 0.0 -2.55',basis='6-31g',charge=-3,spin=5)
m.build()
mf=scf.UHF(m)
mf.kernel()
deltaMulliken=mf.mulliken_pop()
deltaLowdin=mf.mulliken_pop_meta_lowdin_ao()
#print(deltaMulliken[1])
#print(deltaLowdin[1])
print('Fe atom charge')
print('%20s  %.3f'%('Mulliken',deltaMulliken[1][0]))
print('%20s  %.3f'%('Lowdin',deltaLowdin[1][0]))

converged SCF energy = -4019.26973672061  <S^2> = 8.7560837  2S+1 = 6.0020275
 ** Mulliken pop       alpha | beta **
pop of  0 Fe 1s        0.99998 | 0.99998   
pop of  0 Fe 2s        0.99977 | 0.99987   
pop of  0 Fe 3s        1.00268 | 1.00153   
pop of  0 Fe 4s        0.32401 | 0.29096   
pop of  0 Fe 5s       -0.04692 | -0.05245  
pop of  0 Fe 2px       0.99970 | 0.99992   
pop of  0 Fe 2py       0.99970 | 0.99992   
pop of  0 Fe 2pz       0.99970 | 0.99992   
pop of  0 Fe 3px       0.99772 | 0.99951   
pop of  0 Fe 3py       0.99772 | 0.99951   
pop of  0 Fe 3pz       0.99772 | 0.99951   
pop of  0 Fe 4px       0.10430 | 0.09236   
pop of  0 Fe 4py       0.10430 | 0.09236   
pop of  0 Fe 4pz       0.10430 | 0.09236   
pop of  0 Fe 5px       0.01677 | 0.01833   
pop of  0 Fe 5py       0.01677 | 0.01833   
pop of  0 Fe 5pz       0.01677 | 0.01833   
pop of  0 Fe 3dxy      0.74080 | 0.00425   
pop of  0 Fe 3dyz      0.74080 | 0.00425   
pop of  0 Fe 3dz^2     0.75001 | 0.03890   
pop

pop of  3 Cl 4pz       0.00168 | 0.00197   
pop of  4 Cl 1s        1.00000 | 1.00000   
pop of  4 Cl 2s        0.99999 | 0.99999   
pop of  4 Cl 3s        0.98331 | 0.98153   
pop of  4 Cl 4s        0.00206 | 0.00211   
pop of  4 Cl 2px       1.00000 | 1.00000   
pop of  4 Cl 2py       1.00000 | 1.00000   
pop of  4 Cl 2pz       1.00000 | 1.00000   
pop of  4 Cl 3px       0.99737 | 0.99381   
pop of  4 Cl 3py       0.95415 | 0.92278   
pop of  4 Cl 3pz       0.99737 | 0.99381   
pop of  4 Cl 4px       0.00168 | 0.00197   
pop of  4 Cl 4py       0.00336 | 0.00375   
pop of  4 Cl 4pz       0.00168 | 0.00197   
pop of  5 Cl 1s        1.00000 | 1.00000   
pop of  5 Cl 2s        0.99999 | 0.99999   
pop of  5 Cl 3s        0.98331 | 0.98153   
pop of  5 Cl 4s        0.00206 | 0.00211   
pop of  5 Cl 2px       1.00000 | 1.00000   
pop of  5 Cl 2py       1.00000 | 1.00000   
pop of  5 Cl 2pz       1.00000 | 1.00000   
pop of  5 Cl 3px       0.99737 | 0.99381   
pop of  5 Cl 3py       0.99737 |

## Example 3: Bond Orders 
This example computes the H-H bond order of H2 at two different bond lengths. The bond order "should" be 1 near equilibrium and 0 at dissociation, but what happens when this is computed explicitly?

This first block computes the RHF/3-21G orbitals of H2 near equilibrium and for a stretched bond, and writes both as Molden files. 

In [10]:
m=gto.Mole(atom='H 0.0 0.0 0.0; H 0.0 0.0 0.74',basis='3-21g')
m.build()
mf=scf.RHF(m)
mf.kernel()
with open('H2eq.molden', 'w') as f1:
    molden.header(m, f1)
    molden.orbital_coeff(m, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)
m=gto.Mole(atom='H 0.0 0.0 0.0; H 0.0 0.0 2.0',basis='3-21g')
m.build()
mf=scf.RHF(m)
mf.kernel()
with open('H2str.molden', 'w') as f1:
    molden.header(m, f1)
    molden.orbital_coeff(m, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

converged SCF energy = -1.12294025684806
converged SCF energy = -0.90780477393739


## Results 

| Method | Equilibrium | Stretched |
| -- 
| Mayer |1.00000000|1.00000000|
| Wiberg |1.00000000|1.00000000|
| Mulliken |0.80825721|0.33344113|
| FBO |0.99999991|0.99999659|
| LBO |0.737313|<0.05|


In [ ]:
elements = [atom.GetSymbol() for atom in ma2.GetAtoms()]
coordinates = ma2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
ma4 = gto.Mole(basis="3-21g")
ma4.atom = atoms
ma4.build();
mfa=scf.RHF(ma4)
mfa.kernel() 
elements = [atom.GetSymbol() for atom in mn2.GetAtoms()]
coordinates = mn2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
mn4 = gto.Mole(basis="3-21g")
mn4.atom = atoms
mn4.build();
mfn=scf.RHF(mn4)
mfn.kernel() 

Next block generates cubes of density and electrostatic potential 

In [ ]:
cubegen.density(ma4, f'aniline_density.cube', mfa.make_rdm1())
cubegen.mep(ma4, f'aniline_esp.cube', mfa.make_rdm1())
cubegen.density(mn4, f'nitrobenzene_density.cube', mfn.make_rdm1())
cubegen.mep(mn4, f'nitrobenzene_esp.cube', mfn.make_rdm1())

Next block visualizes the densities alone. This isn't very interesting! 

In [ ]:
with open(f"aniline_density.cube") as f:
    da = f.read()
with open(f"nitrobenzene_density.cube") as f:
    dn = f.read()
p = py3Dmol.view(width=400,height=200,viewergrid=(1,2))
p.addModel(ma3, 'sdf',viewer=(0,0))
p.addModel(mn3, 'sdf',viewer=(0,1))
p.addVolumetricData(da, "cube", {'isoval': 0.002, 'color': "gray", 'opacity': 0.75},viewer=(0,0))
p.addVolumetricData(dn, "cube", {'isoval': 0.002, 'color': "gray", 'opacity': 0.75},viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.render()

In [ ]:
with open(f"aniline_esp.cube") as f:
    va = f.read()
with open(f"nitrobenzene_esp.cube") as f:
    vn = f.read()
p = py3Dmol.view(width=400,height=200,viewergrid=(1,2))
p.addModel(ma3, 'sdf',viewer=(0,0))
p.addModel(mn3, 'sdf',viewer=(0,1))
p.addVolumetricData(da, "cube", {'isoval': 0.002, 'opacity': 0.9,'voldata': va, 'volformat':'cube',
      'volscheme': {'gradient':'rwb', 'min':-.05, 'max':.05}},viewer=(0,0))
p.addVolumetricData(dn, "cube", {'isoval': 0.002, 'opacity': 0.9,'voldata': vn, 'volformat':'cube',
      'volscheme': {'gradient':'rwb', 'min':-.05, 'max':.05}},viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.render()

### Example 3: Banana bonds 

In undergraduate chemistry, you learn that H2 has a single bond with covalent bond order 1, two electrons in a bonding orbital. He2 has two electrons in a bonding orbital and two electrons in an antibonding orbital,  covalent bond order 0. The He2 wavefunction can be written as two doubly-occupied atomic orbitals, or as doubly-occupied bonding and antibonding orbitals. Hybridizing the two doubly-occupied atomic orbitals does not change the energy. 

This can become confusing for ethylene. The ethylene wavefunction can be written as doubly occupied sigma and pi bonding orbitals, or as the two doubly occupied linear combinations sigma+pi and sigma-pi. Here you'll view these "banana bonds" and compare to the sigma and pi picture. Both pictures are equivalent, both pictures are equally valid, both are equally "real". 

This first block generates ethylene geometry and orbitals. 

In [ ]:
m=Chem.MolFromSmiles('C=C')
m2=Chem.AddHs(m)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
mb=Chem.MolToMolBlock(m2)
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]

pyscf_mole = gto.Mole(basis="3-21g")
pyscf_mole.atom = atoms
pyscf_mole.build();

mf=scf.RHF(pyscf_mole)
mf.kernel() 
p=py3Dmol.view(width=400,height=400)
p.addModel(mb,'sdf')
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

This block computesthe highest-occupied orbital (pi bonding orbital), the third-highest-occupied bonding orbital (C-C sigma bonding orbital), and their linear combinations 

In [ ]:
cubegen.orbital(pyscf_mole, f'ethylene_pi.cube', mf.mo_coeff[:,7],  nx=60, ny=60, nz=60)
cubegen.orbital(pyscf_mole, f'ethylene_sigma.cube', mf.mo_coeff[:,5],  nx=60, ny=60, nz=60)
cubegen.orbital(pyscf_mole, f'ethylene_banana1.cube', (mf.mo_coeff[:,7]+mf.mo_coeff[:,5])/2**0.5,  nx=60, ny=60, nz=60)
cubegen.orbital(pyscf_mole, f'ethylene_banana2.cube', (mf.mo_coeff[:,7]-mf.mo_coeff[:,5])/2**0.5,  nx=60, ny=60, nz=60)

In [ ]:
p = py3Dmol.view(width=600,height=400,viewergrid=(2,2))
for i in range(2):
    for j in range(2):
        p.addModel(Chem.MolToMolBlock(m2), 'sdf',viewer=(i,j))
        p.setStyle({'stick':{},'sphere':{"scale":0.3}},viewer=(i,j))
with open(f"./ethylene_pi.cube") as f:
    pi = f.read()
p.addVolumetricData(pi, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(0,0))
p.addVolumetricData(pi, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(0,0))
with open(f"./ethylene_sigma.cube") as f:
    sigma= f.read()
p.addVolumetricData(sigma, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(1,0))
p.addVolumetricData(sigma, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(1,0))

with open(f"./ethylene_banana1.cube") as f:
    b1 = f.read()
p.addVolumetricData(b1, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(0,1))
p.addVolumetricData(b1, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(0,1))
with open(f"./ethylene_banana2.cube") as f:
    b2= f.read()
p.addVolumetricData(b2, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(1,1))
p.addVolumetricData(b2, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(1,1))
#p.zoomTo()
#p.update()
p.render()

Your assignment for this module has 3 parts. 

Part 1: Download and install Multiwfn, and use it to compute the hydrogen atom partial charge for methane, using the same level of theory as the H2O calculation above. You should find that, while the value of partial charges differ with different schemes, the difference between partial charges for methane vs. water are consistent with different methods. 

Part 2: Use Multiwfn use it to compute the HF/6-31G partial atomic charge on the iron atom of high-spin [FeCl6](3-) and [Fe(H2O)6](3+) Use the FeCl6 geometry above, and use the following geometry for Fe(H2O)6. Are the changes in partial charge between two complexes reasonable? Explain your reasoning. 

---------------------------------------------------------------------
 Center     Atomic      Atomic             Coordinates (Angstroms)
 Number     Number       Type             X           Y           Z
 ---------------------------------------------------------------------
      1         26           0        0.000218    0.000026   -0.000076
      2          8           0        1.077455    1.729495   -0.205007
      3          1           0        0.857422    2.590219    0.224658
      4          8           0       -1.142871    0.520039   -1.617825
      5          1           0       -1.035324    1.345604   -2.147665
      6          8           0        1.314169   -0.966074   -1.238331
      7          1           0        1.229139   -1.028834   -2.219510
      8          8           0       -1.314543    0.965193    1.238319
      9          1           0       -2.130759    1.435726    0.944618
     10          8           0        1.142649   -0.519670    1.617671
     11          1           0        1.883637    0.017974    1.986101
     12          8           0       -1.077345   -1.729167    0.205280
     13          1           0       -1.892143   -1.829129    0.752952
     14          1           0        2.128900   -1.439136   -0.944574
     15          1           0       -0.860345   -2.589067   -0.227544
     16          1           0        1.035752   -1.345198    2.147691
     17          1           0       -1.228386    1.030324    2.219230
     18          1           0        1.894896    1.829084   -0.748831
     19          1           0       -1.884579   -0.016782   -1.985992


Part 3: Determine the P-O bond order and P and O partial charges of H3PO at the HF/6-31G(d) optimized geometry. Compare at least 4 definitions of partial charge and at least 3 definitions of bond order. 